<a href="https://colab.research.google.com/github/PXamir/RDD_Tranformaciones-Acciones/blob/main/Ejemplos_Acciones_Transformaciones_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instala PySpark si no lo tienes ya instalado
!pip install pyspark

from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("Tranformacion/Accion").getOrCreate()

# Obtener el SparkContext desde SparkSession
sc = spark.sparkContext


In [4]:
######     MAP     ######

# Crear datos
datos_ventas = [
#   (  producto,  cantidad, precio_por_unidad)
    ("Producto A",   3,          10.0),
    ("Producto B",   2,          15.5),
    ("Producto C",   5,          7.2)
]

# Crear el RDD a partir de la lista
rdd_ventas = sc.parallelize(datos_ventas)

# Usar map para calcular el precio total y agregarlo a cada tupla
rdd_ventas_totales = rdd_ventas.map(lambda x: (x[0], x[1], x[2], x[1] * x[2]))

# Mostrar los resultados
print(rdd_ventas_totales.collect())



[('Producto A', 3, 10.0, 30.0), ('Producto B', 2, 15.5, 31.0), ('Producto C', 5, 7.2, 36.0)]
